In [1]:
import os
import sys
import numpy as np

sys.path.append("/home/brent10/opensn/opensn/modules")

from pyopensn.mesh import FromFileMeshGenerator, PETScGraphPartitioner
from pyopensn.xs import MultiGroupXS
from pyopensn.aquad import GLCProductQuadrature2DXY
from pyopensn.solver import DiscreteOrdinatesProblem, PowerIterationKEigenSolver
from pyopensn.context import UseColor, Finalize

UseColor(False)

In [2]:
print(sys.path)

['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/brent10/opensn/lib/python3.12/site-packages', '/home/brent10/opensn/opensn/modules']


In [3]:
casename = '2A'
h5_name = '2a'

## Import Mesh Obj

In [4]:
mesh_filepath = 'lattice_'+casename+'.obj'
meshgen = FromFileMeshGenerator(
    filename=mesh_filepath,
    partitioner=PETScGraphPartitioner(type='parmetis')
)

grid = meshgen.Execute()
grid.ExportToPVTU('mesh_2A')

OpenSn version 0.0.1
2025-04-28 22:04:13 Running OpenSn with 1 processes.

[0]  FromFileMeshGenerator: Generating UnpartitionedMesh
[0]  Making Unpartitioned mesh from wavefront file lattice_2A.obj
[0]  Max material id: 10
[0]  Done checking cell-center-to-face orientations
[0]  00:00:01.3 Establishing cell connectivity.
[0]  00:00:01.3 Vertex cell subscriptions complete.
[0]  00:00:01.3 Surpassing cell 3888 of 38880 (10%)
[0]  00:00:01.3 Surpassing cell 7776 of 38880 (20%)
[0]  00:00:01.3 Surpassing cell 11665 of 38880 (30%)
[0]  00:00:01.3 Surpassing cell 15552 of 38880 (40%)
[0]  00:00:01.3 Surpassing cell 19440 of 38880 (50%)
[0]  00:00:01.3 Surpassing cell 23329 of 38880 (60%)
[0]  00:00:01.3 Surpassing cell 27217 of 38880 (70%)
[0]  00:00:01.3 Surpassing cell 31104 of 38880 (80%)
[0]  00:00:01.3 Surpassing cell 34992 of 38880 (90%)
[0]  00:00:01.3 Surpassing cell 38880 of 38880 (100%)
[0]  00:00:01.3 Establishing cell boundary connectivity.
[0]  00:00:01.3 Done establishing cell 

## Import OpenMC Cross Sections

In [5]:
xs_filepath = 'mgxs_casl_'+h5_name+'/mgxs_'+h5_name+'_one_eighth_SHEM-361.h5'
xs_dict = {}
xs_list = []

h5_mat_names = ['fuel', 'fuel clad', 'fuel gap', 
                'gt-clad', 'gt-water-in', 'gt-water-out', 
                'it-clad', 'it-water-in', 'it-water-out', 
                'moderator', 'water_outside']

for name in h5_mat_names:
    xs_dict[name] = MultiGroupXS()
    xs_dict[name].LoadFromOpenMC(xs_filepath, name, 294.0)
    xs_list = np.append(xs_list,xs_dict[name])

block_ids = [i for i in range(0,len(xs_list))]

scat_order = 1 #xs_list[0].scattering_order

[0]  Reading OpenMC cross-section file "mgxs_casl_2a/mgxs_2a_one_eighth_SHEM-361.h5"
[0]  mgxs_casl_2a/mgxs_2a_one_eighth_SHEM-361.h5 cross-section data evaluated at 294K
[0]  *** WARNING ***  Estimating absorption from the transfer matrices.
[0]  *** WARNING ***  Negative absorption cross section encountered in group 0 when estimating from the transfer matrices
[0]  *** WARNING ***  Negative absorption cross section encountered in group 1 when estimating from the transfer matrices
[0]  Reading OpenMC cross-section file "mgxs_casl_2a/mgxs_2a_one_eighth_SHEM-361.h5"
[0]  mgxs_casl_2a/mgxs_2a_one_eighth_SHEM-361.h5 cross-section data evaluated at 294K
[0]  *** WARNING ***  Estimating absorption from the transfer matrices.
[0]  *** WARNING ***  Negative absorption cross section encountered in group 0 when estimating from the transfer matrices
[0]  *** WARNING ***  Negative absorption cross section encountered in group 1 when estimating from the transfer matrices
[0]  *** WARNING ***  Nega

## Create Quadratures

In [6]:
pquad = GLCProductQuadrature2DXY(32, 4)

## Physics

In [7]:
num_groups = 361
#num_groups = 361
# Constraints: "classic_richardson", "petsc_richardson", "petsc_gmres", "petsc_bicgstab"
group_sets = [{
            "groups_from_to": (0, num_groups - 1),
            "angular_quadrature": pquad,
            "angle_aggregation_num_subsets": 1,
            "inner_linear_method": "classic_richardson",
            "l_abs_tol": 1.0e-6,
            "l_max_its": 300,
            #"gmres_restart_interval": 30
            }]

group_sets = [{
            "groups_from_to": (0, num_groups - 1),
            "angular_quadrature": pquad,
            "angle_aggregation_num_subsets": 1,
            "inner_linear_method": "classic_richardson",
            "l_abs_tol": 1.0e-1,
            "l_max_its": 3,
            #"gmres_restart_interval": 30
            }]

# fix this when automating it stops breaking
bound_conditions = [
                        { 'name' : "xmin", 'type' : "reflecting" },
                        { 'name' : "xmax", 'type' : "reflecting" },
                        { 'name' : "ymin", 'type' : "reflecting" },
                        { 'name' : "ymax", 'type' : "reflecting" },
                        { 'name' : "zmin", 'type' : "reflecting" },
                        { 'name' : "zmax", 'type' : "reflecting" }
                        ]
# fix this when automating it stops breaking
xs_mapping = [
            {'block_ids' : [0],'xs' : xs_list[0]},
            {'block_ids' : [1],'xs' : xs_list[1]},
            {'block_ids' : [2],'xs' : xs_list[2]},
            {'block_ids' : [3],'xs' : xs_list[3]},
            {'block_ids' : [4],'xs' : xs_list[4]},
            {'block_ids' : [5],'xs' : xs_list[5]},
            {'block_ids' : [6],'xs' : xs_list[6]},
            {'block_ids' : [7],'xs' : xs_list[7]},
            {'block_ids' : [8],'xs' : xs_list[8]},
            {'block_ids' : [9],'xs' : xs_list[9]},
            {'block_ids' : [10],'xs' : xs_list[10]}
            ]
#xs_mapping = [
#            {'block_ids' : [0],'xs' : xs_list[0]},
#            {'block_ids' : [1],'xs' : xs_list[1]},
#            {'block_ids' : [2],'xs' : xs_list[2]},
#            {'block_ids' : [3],'xs' : xs_list[3]}
#            ]
#xs_mapping = [{'block_ids' : [0],'xs' : xs_list[0]}]

phys = DiscreteOrdinatesProblem(mesh=grid,
                                num_groups=num_groups,
                                groupsets= group_sets,
                                xs_map=xs_mapping
                                        )
phys.SetOptions(scattering_order=scat_order,
                verbose_inner_iterations=True,
                verbose_outer_iterations=True,
                power_default_kappa=1.0,
                power_normalization=1.0,
                save_angular_flux=False,
                #write_restart_time_interval = 3660,
                #write_restart_path = "2A_restart/2A",
                boundary_conditions = bound_conditions
                                    )


In [8]:
k_solver = PowerIterationKEigenSolver(lbs_problem = phys,
                                      k_tol = 1.0e-8
                                     )
k_solver = PowerIterationKEigenSolver(lbs_problem = phys,
                                      k_tol = 1.0e-0
                                     )

In [9]:
k_solver.Initialize()

[0]  
[0]  Initializing LBS SteadyStateSolver with name: LBSDiscreteOrdinatesProblem
[0]  
[0]  Scattering order    : 1
[0]  Number of Groups    : 361
[0]  Number of Group sets: 1
[0]  
[0]  ***** Groupset 0 *****
[0]  Groups:
[0]      0     1     2     3     4     5     6     7     8     9    10    11 
[0]     12    13    14    15    16    17    18    19    20    21    22    23 
[0]     24    25    26    27    28    29    30    31    32    33    34    35 
[0]     36    37    38    39    40    41    42    43    44    45    46    47 
[0]     48    49    50    51    52    53    54    55    56    57    58    59 
[0]     60    61    62    63    64    65    66    67    68    69    70    71 
[0]     72    73    74    75    76    77    78    79    80    81    82    83 
[0]     84    85    86    87    88    89    90    91    92    93    94    95 
[0]     96    97    98    99   100   101   102   103   104   105   106   107 
[0]    108   109   110   111   112   113   114   115   116   117   118 

In [ ]:
k_solver.Execute()

In [ ]:
fflist = phys.GetScalarFieldFunctionList()

In [ ]:
pitch = 1.26
num_cells = 17

def compute_cell_center(i, j, pitch):
    x_center = (i - 1) * pitch - (num_cells // 2) * pitch
    y_center = (j - 1) * pitch - (num_cells // 2) * pitch
    return x_center, y_center

avoid_pairs = {
    4  : [14, 4],
    12 : [3, 6, 9, 15, 12],
    3  : [6, 12, 9],
    9  : [3, 6, 12, 15, 9],
    14 : [14, 4],
    15 : [6, 12, 9],
    6  : [3, 6, 12, 9, 15]
}

def should_avoid(i, j):
    if i in avoid_pairs:
        for _, value in avoid_pairs[i]:
            if value == j:
                return True
    return False


In [ ]:
val_table = {}


for i in range(0,num_cells):
  val_table[i] = {}
  for j in range(0,num_cells):
      val_table[i][j] = -1
      if not should_avoid(i, j):
      
          x_center, y_center = compute_cell_center(i, j, pitch)
          my_lv = logvol.RCCLogicalVolume.Create({ r = 0.4060, x0 = x_center, y0 = y_center, z0 = -1.0, vz = 2.0 })
          

          xs_fuel_pincell = xs.Get(my_xs["fuel_pincell"])
            
          if xs_fuel_pincell and type(xs_fuel_pincell["sigma_f"]) == "table" then
              sig_f = xs_fuel_pincell["sigma_f"]
          else
              error("sigma_f is not a valid number or does not exist in the table")
          end

          val = 0.
          for g = 1, num_groups do
              ffi = fieldfunc.FFInterpolationCreate(VOLUME)
              fieldfunc.SetProperty(ffi, OPERATION, OP_SUM)
              fieldfunc.SetProperty(ffi, LOGICAL_VOLUME, my_lv)
              fieldfunc.SetProperty(ffi, ADD_FIELDFUNCTION, fflist[g])
              
              fieldfunc.Initialize(ffi)
              fieldfunc.Execute(ffi)
              val_g = fieldfunc.GetValue(ffi)
              val = val + val_g * sig_f[g]
          end
          val_table[i][j] = val
      end
  end
end
